In [2]:
import numpy as np
import matplotlib.pyplot as plt
import simim.siminterface as sim
from simim import constants as sc
from simim.galprops import (
    prop_behroozi_sfr, prop_delooze_cii, 
    prop_li_hi, prop_co_sled, prop_mass_hi,
    prop_halpha, prop_galaxy_survey
)


tng100 = sim.SimHandler('TNG100-1')

rng = np.random.default_rng(1511)

pixel_size = tng100.box_edge_no_h / 450
z_for_modeling = 1.173
tng100_snap = tng100.get_snap_from_z(z_for_modeling)

bins = np.logspace(-3,3,21)
bin_centers = (bins[1:] + bins[:-1]) / 2
k = 2*np.pi * bin_centers

In [3]:
from simim import constants as sc
from simim.galprops import prop_behroozi_sfr, prop_delooze_cii

tng100_snap.make_property(prop_behroozi_sfr, other_kws={'rng': rng}, overwrite=True, rename='sfr_behroozi')
tng100_snap.make_property(prop_delooze_cii, other_kws={'rng': rng}, overwrite=True, rename='LCII', kw_remap={'sfr': 'sfr_behroozi'})

d = tng100_snap.cosmo.comoving_distance(z_for_modeling).value
yfactor = sc.c / sc.nu_cii * (1+z_for_modeling)**2 / (1000*tng100_snap.cosmo.H(z_for_modeling).value)
dl = (1+z_for_modeling)*d

convnucii = sc.Lsun_to_W / (4*np.pi*dl**2) * d**2 * yfactor / pixel_size**3 \
    * 1 / 3.0857e22**2 *1e26 # Convert to Jy/sr from Lsun/Mpc^3

grid_CII = tng100_snap.grid('LCII', res=pixel_size, norm=convnucii)

/Users/shubh/Library/CloudStorage/OneDrive-PennO365/TIM/crossLIM/simim_public/simim/_handlers.py:312: UserWarning: Property sfr_behroozi already exists, overwriting
  warnings.warn("Property {} already exists, overwriting".format(name))
/Users/shubh/Library/CloudStorage/OneDrive-PennO365/TIM/crossLIM/simim_public/simim/_handlers.py:312: UserWarning: Property LCII already exists, overwriting
  warnings.warn("Property {} already exists, overwriting".format(name))


In [4]:
from simim.galprops import Prop, prop_li_hi, prop_mass_hi
from astropy.cosmology import Planck18 as cosmo
from astropy import units as u, constants as con

nu21rest = 1.420405751e9  # Hz
nu21obs = nu21rest / (1 + z_for_modeling)  # Hz
conv_mK = 1e3 * (sc.c**2 / (2 * con.k_B * nu21obs**2)).to(u.K/u.J).value * 1e-26  # Jy/sr to mK

d = tng100_snap.cosmo.comoving_distance(z_for_modeling).value
yfactor = sc.c / nu21rest * (1+z_for_modeling)**2 / (1000*tng100_snap.cosmo.H(z_for_modeling).value)
dl = (1+z_for_modeling)*d

conv21 = sc.Lsun_to_W / (4*np.pi*dl**2) * d**2 * yfactor / pixel_size**3 \
    * 1 / 3.0857e22**2 *1e26

tng100_snap.make_property(prop_mass_hi, other_kws={'rng': rng, 'cosmo': cosmo}, overwrite=True)
tng100_snap.make_property(prop_li_hi, overwrite=True)

grid_HI = tng100_snap.grid('LHI', res=pixel_size, norm=conv21 * conv_mK)

In [5]:
from simim.galprops import prop_co_sled
from simim import constants as sc

tng100_snap.make_property(prop_co_sled, other_kws={'rng': rng, 'fircor':0.6}, overwrite=True, kw_remap={'sfr': 'sfr_behroozi'})

nu_co = sc.nu_co43
lco = 'L54'

d = tng100_snap.cosmo.comoving_distance(z_for_modeling).value
yfactor = sc.c / nu_co * (1+z_for_modeling)**2 / (1000*tng100_snap.cosmo.H(z_for_modeling).value)
dl = (1+z_for_modeling)*d
convnuco = sc.Lsun_to_W / (4*np.pi*dl**2) * d**2 * yfactor / pixel_size**3 \
    * 1 / 3.0857e22**2 *1e26 # Convert to Jy/sr from Lsun/Mpc^3

grid_CO = tng100_snap.grid(lco, res=pixel_size, norm=convnuco)

In [13]:
from simim.galprops import prop_halpha, prop_galaxy_survey

tng100_snap.make_property(prop_halpha, other_kws={'rng': rng}, overwrite=True, kw_remap={'sfr': 'sfr_behroozi'})
tng100_snap.make_property(prop_galaxy_survey, other_kws={'fluxcut': 6.9e-17, 'NIIcorr':0.4}, overwrite=True)

grid_gal = tng100_snap.grid('ngal', res=pixel_size)
grid_gal.grid /= np.mean(grid_gal.grid)
grid_gal.grid -= 1.0

In [ ]:
# save all the grids
np.savez('tng100_z1p173_grids.npz', 
        grid_CII=grid_CII.grid, 
        grid_HI=grid_HI.grid, 
        grid_CO=grid_CO.grid, 
        grid_gal=grid_gal.grid)